# Kapitel 6 – Funktionen

Dieses Notebook begleitet das Kapitel **Funktionen**. Es verwendet **synthetische Beispiele** (keine externen Daten).


## Lernziele
- Zweck und Nutzen von Funktionen verstehen
- Parameter/Argumente/Rückgabewerte unterscheiden
- Eigene Funktionen definieren, dokumentieren und testen
- Positions-, benannte und variable Argumente verwenden
- Rekursion an einem einfachen Beispiel nachvollziehen
- (Exkurs) Encodings praktisch nachvollziehen


## Exkurs: Datei-Encodings (ASCII vs. UTF‑8)
Wir demonstrieren das Encodieren/Decodieren und was bei falschem Encoding passiert.

In [ ]:

# ASCII-Beispiel (Latein – ohne Sonderzeichen funktioniert)
t_lat = "Gallia est omnis divisa in partes tres."
ascii_bytes = t_lat.encode('ascii')
ascii_codes = list(ascii_bytes)
ascii_codes[:20], len(ascii_codes)


In [ ]:

# Altgriechisch – ASCII schlägt fehl, UTF-8 funktioniert
t_gr = "Δαρείου καὶ Παρυσάτιδος γίγνονται παῖδες δύο"
try:
    _ = t_gr.encode('ascii')
except UnicodeEncodeError as e:
    print("ASCII-Fehler:", e)

utf8_bytes = t_gr.encode('utf-8')
codes_utf8 = list(utf8_bytes)[:32]
print("UTF-8: erste 32 Byte:", codes_utf8)

# Korrektes Decodieren
print(bytes(utf8_bytes).decode('utf-8'))


In [ ]:

# Falsches Decodieren (UTF-8-Bytes als Latin-1 interpretieren) – ergibt Kauderwelsch
latin1_wrong = bytes(utf8_bytes).decode('latin1')
latin1_wrong[:120]


## Funktionen in Python – Definition, Signatur, Aufruf

In [ ]:

# Eine einfache Funktion: Länge eines Strings zählen (ohne len())
def string_length(our_string):
    """Compute the length of a string.

Args:
    our_string (str): String to measure.

Returns:
    int: Number of characters (incl. spaces).
    """
    counter = 0
    for _ in our_string:
        counter += 1
    return counter

name = "Gaius Iulius Caesar"
print("Eigene Funktion:", string_length(name))
print("len():          ", len(name))
print("Docstring:\n", string_length.__doc__)


In [ ]:

# Variante mit Typannotationen
from typing import Iterable

def string_length_typed(our_string: str) -> int:
    counter = 0
    for _ in our_string:
        counter += 1
    return counter

import inspect
print("Signatur:", inspect.signature(string_length_typed))
print("Ergebnis:", string_length_typed("Cicero"))


## Argumentarten

### 1) Positionsargumente

In [ ]:

def roman_name(praenomen, nomen_gentile, cognomen):
    return f"{praenomen} {nomen_gentile} {cognomen}"

print(roman_name("Marcus", "Tullius", "Cicero"))
print(roman_name("Tullius", "Cicero", "Marcus"))  # falsche Reihenfolge


### 2) Benannte Argumente (mit Defaults)

In [ ]:

def roman_name_kw(praenomen: str = "", nomen_gentile: str = "", cognomen: str = "") -> str:
    return f"{praenomen} {nomen_gentile} {cognomen}".strip()

print(roman_name_kw(praenomen="Marcus", nomen_gentile="Tullius", cognomen="Cicero"))
print(roman_name_kw(nomen_gentile="Tullius", praenomen="Marcus"))  # ohne Cognomen


### 3) Variable Argumente (`*args`, `**kwargs`)

In [ ]:

def roman_name_var(*names, sep: str = " "):
    """Beliebig viele Namenselemente zusammenfügen.

Args:
    *names: beliebige Namenssegmente
    sep (str): Trennzeichen (benanntes Argument)
"""
    return sep.join(str(n) for n in names if n)

print(roman_name_var("Gaius", "Iulius", "Caesar"))
print(roman_name_var("Publius", "Vergilius", "Maro", sep=" · "))

def summarize_person(**facts):
    """Sammelt beliebige Fakten als benannte Argumente."""
    return facts

summarize_person(praenomen="Marcus", gens="Tullius", cognomen="Cicero", born=-106, died=-43)


## Mini‑Aufgabe: Datumsangaben normalisieren
Schreibe eine Funktion `normalize_year(s)`, die Jahresangaben wie `'0259'`, `'003'`, `'-0003'` in `int` wandelt (BC als negative Werte).

In [ ]:

def normalize_year(s: str) -> int | None:
    """Normalisiert Jahresstrings zu int.

Beispiele:
    '0259' -> 259
    '0003' -> 3
    '-0003' -> -3
    None/'' -> None
    """
    if s is None:
        return None
    s = str(s).strip()
    if not s:
        return None
    # BC-Fall
    sign = -1 if s.startswith('-') else 1
    s = s.lstrip('+-')  # + oder - entfernen
    if not s.isdigit():
        return None
    val = int(s.lstrip('0') or '0')
    return sign * val

tests = ['0259', '0003', '-0003', '0', '', None, '+0012', ' 0010 ']
[(t, normalize_year(t)) for t in tests]


## Funktionen kombinieren
Beispiel: Namen formatieren und Datumsangaben ausgeben.

In [ ]:

def format_person(praenomen="", nomen="", cognomen="", born=None, died=None):
    name = roman_name_var(praenomen, nomen, cognomen)
    y_b = normalize_year(born)
    y_d = normalize_year(died)
    life = ""
    if y_b is not None or y_d is not None:
        life = f" ({y_b if y_b is not None else '?'}–{y_d if y_d is not None else '?'})"
    return name + life

format_person("Marcus", "Tullius", "Cicero", born='-0106', died='-0043')


## Rekursivität

### Beispiel 1: Fakultät (n!) – mit Guard Clauses und Typannotationen

In [ ]:

def factorial(n: int) -> int:
    if not isinstance(n, int):
        raise TypeError("n muss int sein")
    if n < 0:
        raise ValueError("n muss ≥ 0 sein")
    if n in (0, 1):
        return 1
    return n * factorial(n-1)

[factorial(i) for i in range(7)]


### Beispiel 2: Rekursives Durchlaufen eines verschachtelten Datentyps

In [ ]:

# Wir traversieren ein verschachteltes Dict (synthetisch) und sammeln alle Schlüsselpfade
data = {
    "coin_6": {"coin_no": 6, "weight": 5.41, "issuer": {"name": "Claudius I", "house": "Claudii"}},
    "coin_4": {"coin_no": 4, "weight": 6.49, "issuer": {"name": "Agrippa (Gaius)", "house": None}},
}

def collect_keypaths(obj, prefix=()):
    paths = []
    if isinstance(obj, dict):
        for k, v in obj.items():
            paths.extend(collect_keypaths(v, prefix + (k,)))
    else:
        paths.append((prefix, obj))
    return paths

collect_keypaths(data)[:6]



## Gute Praxis (Checkliste)
- **Eine Aufgabe pro Funktion** – kleine, klar benannte Einheiten (`normalize_date()`, `extract_names()`).
- **Sprechende Namen** – Verben für Funktionen, Nomen für Daten.
- **Benannte Argumente & Defaults** – erhöhen Lesbarkeit/Robustheit.
- **Dokumentieren** – knapper Docstring (Zweck, Args, Return).
- **Rückgaben klar** – eindeutige Typen; `None` bewusst behandeln.
- **Guard Clauses** – `if x is None: return …` verkürzt verschachtelte Logik.



## Übungsaufgaben (optional)
1. **`slugify(name: str) -> str`**: Schreibe eine Funktion, die einen lateinischen Namen in einen URL‑Slug umwandelt (kleinbuchstaben, Leerzeichen→`-`, Sonderzeichen entfernen).
2. **`findall_years(text: str) -> list[int]`**: Extrahiere aus einem Text alle Jahreszahlen (2–4 Stellen, optional `-` davor) und gib sie als `int` zurück – nutze eine kleine Hilfsfunktion und `normalize_year`.
3. **`roman_name_var` erweitern**: Füge ein benanntes Argument `order=("praenomen","nomen","cognomen")` hinzu und erlaube flexible Reihenfolgen.
